<a href="https://colab.research.google.com/github/adampotton/MDM3-Rep-3/blob/main/inception_v3_aerial_trianing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mounting Google Drive and imports

In [2]:
from google.colab import files, drive
drive.mount('/content/drive')
import os
import numpy as np

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Loading Numpy Data Arrays

In [3]:
#if you get an error the first time just run it again
os.chdir('../../../')
!pwd
os.chdir('content')
os.chdir(r'drive/MyDrive/Aerial Data')
print(os.listdir())

train_labels = np.load('train_labels.npy')
train_images = np.load('train_images.npy')
test_labels = np.load('test_labels.npy')
test_images = np.load('test_images.npy')
val_labels = np.load('val_labels.npy')
val_images = np.load('val_images.npy')

/
['aerial_pseudotsuga_menziesii', 'aerial_picea_abies', 'aerial_betula_spec', 'aerial_abies_alba', 'aerial_pinus_nigra', 'aerial_larix_decidua', 'aerial_larix_kaempferi', 'aerial_pinus_strobus', 'aerial_pinus_sylvestris', 'aerial_fraxinus_excelsior', 'aerial_fagus_sylvatica', 'aerial_acer_pseudoplatanus', 'aerial_quercus_robur', 'aerial_quercus_petraea', 'aerial_quercus_rubra', 'aerial_tilia_spec', 'aerial_populus_spec', 'aerial_prunus_spec', 'aerial_alnus_spec', 'train_images.npy', 'train_labels.npy', 'test_images.npy', 'test_labels.npy', 'val_images.npy', 'val_labels.npy']


# If Not using colab the following can be used to import the data

In [4]:
import gdown

gdown.download(r'https://drive.google.com/file/d/1-7lEmQDmf06beJUD3NaQk82125NGqYpm/view?usp=drive_link', r'../../data/aerial_data/val_labels.npy',fuzzy=True)
gdown.download(r'https://drive.google.com/file/d/1-54Z1sN8uYgNG66MwWumRTlYMJmX7wWw/view?usp=drive_link', r'../../data/aerial_data/val_images.npy',fuzzy=True)
gdown.download(r'https://drive.google.com/file/d/1vKs-th9eHU_oMXTasVm-FD9Dximer82Q/view?usp=drive_link', r'../../data/aerial_data/train_labels.npy',fuzzy=True)
gdown.download(r'https://drive.google.com/file/d/1aJkI-7d5KCAAYCoXkrEcHVXvSsXc_-8n/view?usp=drive_link', r'../../data/aerial_data/train_images.npy',fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1-7lEmQDmf06beJUD3NaQk82125NGqYpm
To: c:\Users\bench\OneDrive\Documents\GitHub\MDM3-Rep-3\data\aerial_data\val_labels.npy
100%|██████████| 38.9k/38.9k [00:00<?, ?B/s]
Downloading...
From (original): https://drive.google.com/uc?id=1-54Z1sN8uYgNG66MwWumRTlYMJmX7wWw
From (redirected): https://drive.google.com/uc?id=1-54Z1sN8uYgNG66MwWumRTlYMJmX7wWw&confirm=t&uuid=510fa68d-caf5-408a-9a59-45cf796a8dc1
To: c:\Users\bench\OneDrive\Documents\GitHub\MDM3-Rep-3\data\aerial_data\val_images.npy
100%|██████████| 171M/171M [00:14<00:00, 11.6MB/s] 
Downloading...
From (original): https://drive.google.com/uc?id=1aJkI-7d5KCAAYCoXkrEcHVXvSsXc_-8n
From (redirected): https://drive.google.com/uc?id=1aJkI-7d5KCAAYCoXkrEcHVXvSsXc_-8n&confirm=t&uuid=337c523f-4708-43a3-8990-a3ffb1a8c052
To: c:\Users\bench\OneDrive\Documents\GitHub\MDM3-Rep-3\data\aerial_data\train_labels.npy
100%|██████████| 1.36G/1.36G [01:57<00:00, 11.6MB/s]
Downloading...
From: https://dri

# Loading numpy files from memory

In [24]:
import os
import numpy as np
#system specific
folder = '../../data/aerial_data'
train_images = np.load(os.path.join(folder, 'train_images.npy'))
train_labels = np.load(os.path.join(folder, 'train_labels.npy'))
val_images = np.load(os.path.join(folder, 'val_images.npy'))
val_labels = np.load(os.path.join(folder, 'val_labels.npy'))

In [25]:
#inspecting data
print("train data:")
print("images shape:")
print(train_images.shape)
print("labels shape:")
print(train_labels.shape)

#print("test data:")
#print("images shape:")
#print(test_images.shape)
#print("labels shape:")
#print(test_labels.shape)

print("val data:")
print("images shape:")
print(val_images.shape)
print("labels shape:")
print(val_labels.shape)

print("image format check:")
print(train_images[0,:,:,0])

print("label check:")
print(train_labels[0])


train data:
images shape:
(3688, 304, 304, 4)
labels shape:
(3688,)
val data:
images shape:
(462, 304, 304, 4)
labels shape:
(462,)
image format check:
[[ 55  50  54 ...  79  85  78]
 [ 58  55  51 ...  86  88  83]
 [ 58  60  51 ...  89  82  83]
 ...
 [165 175 173 ... 119 105 111]
 [172 169 167 ... 114 116 123]
 [174 170 162 ... 128 131 133]]
label check:
Fagus_sylvatica


# Performing Feature Extraction on Inception-V3, which is currently trained on image net
Adapted from: <https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/finetuning_torchvision_models_tutorial.ipynb#scrollTo=NiFT4EYmprqG>

# Training Parameters

In [26]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

# Number of tree species categories
num_classes = 19

# Batch size for training (should be a power of 2 if training on the GPU and is limited by memory)
batch_size = 32

# Number of epochs to train for
num_epochs = 30

#True if we are just training the output layer of the network
feature_extract = True

print("cuda is available = ",torch.cuda.is_available())

cuda is available =  True


# Function For training the model

In [27]:
from torch.utils.data import SummaryWriter
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=True, use_tensorboard=False):
    if use_tensorboard:
        #initialize tensorboard writer
        writer = SummaryWriter()
    
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        if use_tensorboard:
                            writer.add_scalar('Training Loss', loss, epoch)
                            writer.add_scalar('Training Accuracy', torch.sum(preds == labels.data).double() / len(labels), epoch)
                        
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            #deep copying the model if its performance has improved
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)
                if use_tensorboard:
                    writer.add_scalar('Validation Accuracy', epoch_acc, epoch)
                    writer.add_scalar('Validation Loss', epoch_loss, epoch)

        print("epoch {} complete".format(epoch))

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    if use_tensorboard:
        #making sure all data is written to disk
        writer.flush()
    
    # load best model weights and return best model
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

# Ensuring that model parameters are not used during training since we are only interested in the final layer

In [28]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

# Defining function to reshape the model to account for the number of classes
By default Inception-v3 has 1000 classes as it is trained on imageNet, whilst our dataset has 19

In [29]:

def initialize_inception_model(num_classes, feature_extract, use_pretrained=True, model=None):
  # Initialize these variables which will be set in this if statement. Each of these
  #   variables is model specific.
  model_ft = None
  input_size = 0

  """ Inception v3
  Be careful, expects (299,299) sized images and has auxiliary output
  """
  if model is None:
    if use_pretrained:
      model_ft = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', weights='Inception_V3_Weights.IMAGENET1K_V1')
    else:
      model_ft = models.inception_v3('pytorch/vision:v0.10.0', 'inception_v3')
  else:
    model_ft = model
  #"switching off" training on the majority of the model weights if feature_extract
  set_parameter_requires_grad(model_ft, feature_extract)
  # Handle the auxilary net
  num_ftrs = model_ft.AuxLogits.fc.in_features
  model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
  # Handle the primary net
  num_ftrs = model_ft.fc.in_features
  model_ft.fc = nn.Linear(num_ftrs, num_classes)
  input_size = 299
  #adding an initial 1x1 convolutional layer to handle 4 channel input and reduce it to 3 channels
  input_embedding_module = nn.Sequential(nn.Conv2d(4, 3, kernel_size=1))#keeping this module seperate for future use
  #wrapping the pretrained modek and the input embedding module together for a combined model
  model_ft = nn.Sequential(input_embedding_module, model_ft)

  return model_ft, input_size


# Initializing Model


In [30]:
#initializing model
#Initialize the model for this run
model_ft, input_size = initialize_inception_model(num_classes, feature_extract, use_pretrained=True)
print(model_ft)

Using cache found in C:\Users\bench/.cache\torch\hub\pytorch_vision_v0.10.0


Sequential(
  (0): Sequential(
    (0): Conv2d(4, 3, kernel_size=(1, 1), stride=(1, 1))
  )
  (1): Inception3(
    (Conv2d_1a_3x3): BasicConv2d(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_2a_3x3): BasicConv2d(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_2b_3x3): BasicConv2d(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (Conv2d_3b_1x1): BasicConv2d(
      (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(80, eps=0.001, momentum=0

# Formatting Loaded Data and creating data loader

In [31]:
from PIL import Image
#converting numpy arrays to PIL images
train_images_pil = [Image.fromarray(image) for image in train_images]
val_images_pil = [Image.fromarray(image) for image in val_images]
#creating one hot encoding labels from numpy arrays
train_categories, train_labels_numeric = np.unique(train_labels, return_inverse=True)
val_categories, val_labels_numeric = np.unique(val_labels, return_inverse=True)

print("Train Categories:", train_categories)
print("Train Labels Numeric:", train_labels_numeric[0:10])
print("Val Categories:", val_categories)
print("Val Labels Numeric:", val_labels_numeric[0:10])

Train Categories: ['Abies_alba' 'Acer_pseudoplatanus' 'Alnus_spec.' 'Betula_spec.'
 'Fagus_sylvatica' 'Fraxinus_excelsior' 'Larix_decidua' 'Larix_kaempferi'
 'Picea_abies' 'Pinus_nigra' 'Pinus_strobus' 'Pinus_sylvestris'
 'Populus_spec.' 'Prunus_spec.' 'Pseudotsuga_menziesii' 'Quercus_petraea'
 'Quercus_robur' 'Quercus_rubra' 'Tilia_spec.']
Train Labels Numeric: [ 4 12  8 17  4  4 14  3  8 11]
Val Categories: ['Abies_alba' 'Acer_pseudoplatanus' 'Alnus_spec.' 'Betula_spec.'
 'Fagus_sylvatica' 'Fraxinus_excelsior' 'Larix_decidua' 'Larix_kaempferi'
 'Picea_abies' 'Pinus_nigra' 'Pinus_strobus' 'Pinus_sylvestris'
 'Populus_spec.' 'Prunus_spec.' 'Pseudotsuga_menziesii' 'Quercus_petraea'
 'Quercus_robur' 'Quercus_rubra' 'Tilia_spec.']
Val Labels Numeric: [ 3  2  4 14  8  2  4  2  2  8]


In [32]:
if np.all(train_categories == val_categories):
    print("categories are consistent between train and val sets")
    categories = train_categories = val_categories
else:
    raise ValueError("Train and val categories don't match")

categories are consistent between train and val sets


In [33]:
# normalization for training and validation
# right now validation and training transform is the same and no augmentation
# Calculate mean and std for 4-channel images
#TODO: validate that this is correct (currently quite computationally expensive)
mean = np.mean(train_images, axis=(0, 1, 2)) / 255  # divide by 255 to normalize between 0 and 1
std = np.std(train_images, axis=(0, 1, 2)) / 255

# Update the normalization in the transform
image_transform = transforms.Compose([
    transforms.Resize(input_size),
    transforms.CenterCrop(input_size),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)])

#creating a custom pytorch dataset for the numpy arrays we loaded earlier
#with applied transforms
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        if self.transform:
          image = self.transform(image)
          

        return image, label


#creating training and validation pytorch datasets
training_dataset = CustomDataset(train_images_pil, train_labels_numeric, image_transform)
val_dataset = CustomDataset(val_images_pil, val_labels_numeric, image_transform)

# Create training and validation dataloaders
dataloaders_dict = {'train': torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, shuffle=True),
                    'val': torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)}


# Training the model

In [34]:
## Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("sending model to",device)
# Send the model to GPU
model_ft = model_ft.to(device)

# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

#Creating the Optimizer
if feature_extract:
  params_to_update = []
  for name,param in model_ft.named_parameters():
    if param.requires_grad==True:
      params_to_update.append(param)
      print("\t",name)
else:
  params_to_update = model_ft.parameters()

print("number of parameters to train =",len(params_to_update))
for i, param in enumerate(params_to_update):
  print("parameter {}:".format(i),param.shape)


optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)







sending model to cuda:0
	 0.0.weight
	 0.0.bias
	 1.AuxLogits.fc.weight
	 1.AuxLogits.fc.bias
	 1.fc.weight
	 1.fc.bias
number of parameters to train = 6
parameter 0: torch.Size([3, 4, 1, 1])
parameter 1: torch.Size([3])
parameter 2: torch.Size([19, 768])
parameter 3: torch.Size([19])
parameter 4: torch.Size([19, 2048])
parameter 5: torch.Size([19])


In [36]:
# Train and evaluate
model_ft, hist = train_model(model_ft,
                             dataloaders_dict,
                             criterion,
                             optimizer_ft,
                             num_epochs=num_epochs,
                             is_inception=True)

torch.save(model_ft.state_dict(),'training_run_2_4_channels.pth')

Epoch 0/29
----------
train Loss: 3.2352 Acc: 0.3126
val Loss: 2.2454 Acc: 0.3528
epoch 0 complete
Epoch 1/29
----------
train Loss: 3.0908 Acc: 0.3305
val Loss: 2.1837 Acc: 0.3636
epoch 1 complete
Epoch 2/29
----------
train Loss: 2.9935 Acc: 0.3566
val Loss: 2.1398 Acc: 0.3636
epoch 2 complete
Epoch 3/29
----------
train Loss: 2.9195 Acc: 0.3690
val Loss: 2.0747 Acc: 0.3853
epoch 3 complete
Epoch 4/29
----------
train Loss: 2.8608 Acc: 0.3736
val Loss: 2.0351 Acc: 0.4091
epoch 4 complete
Epoch 5/29
----------
train Loss: 2.8085 Acc: 0.3853
val Loss: 2.0244 Acc: 0.4048
epoch 5 complete
Epoch 6/29
----------
train Loss: 2.7756 Acc: 0.3902
val Loss: 2.0033 Acc: 0.4026
epoch 6 complete
Epoch 7/29
----------
train Loss: 2.7361 Acc: 0.3959
val Loss: 1.9954 Acc: 0.4048
epoch 7 complete
Epoch 8/29
----------
train Loss: 2.6966 Acc: 0.3956
val Loss: 1.9713 Acc: 0.4069
epoch 8 complete
Epoch 9/29
----------
train Loss: 2.6763 Acc: 0.4005
val Loss: 1.9559 Acc: 0.4221
epoch 9 complete
Epoch 10/2

# Visualizing the trained model with tensorboard

In [ ]:
!tensorboard --logdir=runs

# Testing Trained Model

### Downloading test data

In [25]:
import gdown

gdown.download(r'https://drive.google.com/file/d/1-4ZKGtGrWCEfaRDBobEgeQJmtdJIFobi/view?usp=drive_link', r'../../data/aerial_data/test_labels.npy',fuzzy=True)
gdown.download(r'https://drive.google.com/file/d/1-01C3LjzGSz7QGzvharc52yZRwEgMzni/view?usp=drive_link', r'../../data/aerial_data/test_images.npy',fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1-4ZKGtGrWCEfaRDBobEgeQJmtdJIFobi
To: c:\Users\bench\OneDrive\Documents\GitHub\MDM3-Rep-3\data\aerial_data\test_labels.npy
100%|██████████| 38.9k/38.9k [00:00<00:00, 2.42MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1-01C3LjzGSz7QGzvharc52yZRwEgMzni
From (redirected): https://drive.google.com/uc?id=1-01C3LjzGSz7QGzvharc52yZRwEgMzni&confirm=t&uuid=f9c3ae1e-174b-4b63-bf67-abcafe428388
To: c:\Users\bench\OneDrive\Documents\GitHub\MDM3-Rep-3\data\aerial_data\test_images.npy
100%|██████████| 170M/170M [00:21<00:00, 7.83MB/s] 


'../../data/aerial_data/test_images.npy'

### Loading Test Data

In [37]:
import numpy as np
from PIL import Image
import os
print(os.getcwd())
test_labels = np.load('../../data/aerial_data/test_labels.npy')
test_images = np.load('../../data/aerial_data/test_images.npy')

#converting numpy arrays into RGB PIL images, ignoring the IR channel for now
test_images_pil = [Image.fromarray(image) for image in test_images]
#creating one hot encoding labels from numpy arrays
test_categories, test_categories_numeric = np.unique(test_labels, return_inverse=True)

print("Test Categories:", test_categories)
print("train Categories:", train_categories)

test_dataset = CustomDataset(test_images_pil, test_categories_numeric, image_transform)

test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

c:\Users\bench\OneDrive\Documents\GitHub\MDM3-Rep-3\training\aerial_data_training
Test Categories: ['Abies_alba' 'Acer_pseudoplatanus' 'Alnus_spec.' 'Betula_spec.'
 'Fagus_sylvatica' 'Fraxinus_excelsior' 'Larix_decidua' 'Larix_kaempferi'
 'Picea_abies' 'Pinus_nigra' 'Pinus_strobus' 'Pinus_sylvestris'
 'Populus_spec.' 'Prunus_spec.' 'Pseudotsuga_menziesii' 'Quercus_petraea'
 'Quercus_robur' 'Quercus_rubra' 'Tilia_spec.']
train Categories: ['Abies_alba' 'Acer_pseudoplatanus' 'Alnus_spec.' 'Betula_spec.'
 'Fagus_sylvatica' 'Fraxinus_excelsior' 'Larix_decidua' 'Larix_kaempferi'
 'Picea_abies' 'Pinus_nigra' 'Pinus_strobus' 'Pinus_sylvestris'
 'Populus_spec.' 'Prunus_spec.' 'Pseudotsuga_menziesii' 'Quercus_petraea'
 'Quercus_robur' 'Quercus_rubra' 'Tilia_spec.']


### evaluating on test data

In [39]:
import torch
import torch.nn.functional as F

model_file = 'training_run_2_4_channels.pth'
trained_model, _ = initialize_inception_model(num_classes, feature_extract, use_pretrained=True)
trained_model.load_state_dict(torch.load(model_file))
trained_model.to(device)
trained_model.eval()

correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for images, labels in test_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = trained_model(images)
        #applying softmax to model logits. This is done implicitley during the training but not during evaluation
        outputs = F.softmax(outputs, dim=1)

        _, predicted = torch.max(outputs, 1)

        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

accuracy = correct_predictions / total_predictions
print("Number of test predictions:", total_predictions)
print(f"Accuracy on the test dataset: {accuracy:.2%}")


Using cache found in C:\Users\bench/.cache\torch\hub\pytorch_vision_v0.10.0


Number of test predictions: 461
Accuracy on the test dataset: 39.26%


In [53]:
print(trained_model)

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri